# **기말 프로젝트 보고서**
**2019320139 최윤지**



# **1. 실험 개요** 
</br>

본 실험은 고객의 Bank Marketing에 대한 데이터를 가지고 고객이 정기예금에 가입할 것인지를 예측하는 Classification 모델을 설계하는 것이다. 
</br>

모델의 경우,

> 1) Deicision Tree  
2) Random Forest  
3) MLP  
4) KNN  
5) Adaboost

</br>

다음과 같이 총 5개를 적용해보았으며, 각각의 알고리즘에 대해 파라미터를 조정하여 여러개의 모델을 만들고, cross-validation score 값을 사용하여 그 중 최적의 모델을 선정하였다. 선정 된 모델을 가지고 test-set을 이용한 결과를 분석해보고, feature selection을 적용한 후에는 어떤 성능 차이를 보이는지도 실험해보았다.  
결론적으로 Bank Marketing의 데이터에 대해 어떤 알고리즘, 어떤 파라미터를 적용한 모델이 가장 좋은 성능을 보였는지 확인하고 그 결과를 분석해볼 것이다.

</br>


---





# **2. 데이터 선정** 

</br>

### **데이터 설명**

실험에 사용 된 데이터는 UCI Machine Learning Repository 에서 가져온 Bank Marketing Data이다. 
(http://archive.ics.uci.edu/ml/datasets/Bank+Marketing)

해당 데이터의 경우 classification을 통해 예측할만한 label 값이 잘 드러나 있어서 선정하게 되었으며, 데이터의 attribute은 다음과 같이 총 16개이다.
</br>


1.  **age (나이)**: numeric
2.  **job (직업)**:  categorical ('admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self-employed', 'services', 'student', 'technician', 'unemployed')
3. **marital(결혼 상태)**: categorical ('divorced', 'married', 'single')
4. **education(교육 수준)**: categorical ('basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate', 'professional.course', 'university.degree', 'unknown')
5. **default(채무불이행 여부)**: categorical ('yes', 'no')
6. **balance(잔고)**: numeric
7. **housing(주택 대출 유무)**: cateogorical ('yes', 'no')
8. **loan(개인 대출 유무)**: categorical ('yes', 'no')
9. **contact(연락처 종류)**: cateogorical('cellular', 'telephone')
10. **month(마지막으로 연락한 달)**: categorical('jan', 'feb', 'mar', ..., 'nov', 'dec')
11. **day_of_week(마지막으로 연락한 요일)**: categorical('mon','tue','wed','thu','fri')
12. **duration(마지막 연락의 지속시간)**: numeric
13. **campaign(이 캠페인에 대해 해당 고객에게 한 연락의 수)**: numeric
14. **pdays(지난 연락 이후 시간)**: numeric
15. **previous(이 캠페인 이전에 해당 고객에게 한 연락의 수)**: numeric
16. **poutcome(이전 마케팅 캠페인의 결과)**: categorical( 'failure', 'nonexistent', 'success')

</br>


17.   **y(해당 고객이 정기예금을 드는지의 여부)**: binary ('yes', 'no')

</br></br>
이 데이터 셋을 다음과 같이 깃허브를 통해 구분자 ';'로 불러왔다. 





In [1]:
import pandas as pd

data_url = "https://raw.githubusercontent.com/yunz0926/COSE362/master/BankMarketingData.csv"

data = pd.read_csv(data_url, sep=';')
data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

해당 데이터셋을 분석해본 결과, education, contact, poutcome, marital attribute에서 unknown 값이 자주 출현했는데 이 값은 NA로 비슷하게 거의 의미가 없어 모델의 성능에 좋지 않은 영향을 미칠 것이라고 생각하였다. 따라서 다음과 같이 'unknown' 값을 가진 instance들은 데이터 셋에서 제거하였다.

In [2]:
data = data[data.education != 'unknown']
data = data[data.contact != 'unknown']
data = data[data.poutcome != 'unknown']
data = data[data.marital != 'unknown']

data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

### **데이터 전처리**
선정 된 데이터 셋에는 범주형 데이터가 많이 포함되어있다. 따라서 sklearn.preprocessing 라이브러리의 LabelEnconder를 사용하여 범주형 attribute들을 수치형으로 변환해주었고, NA 값을 갖는 attribute이 있는지 확인해주었다.

In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data.job = le.fit_transform(data.job)
data.marital = le.fit_transform(data.marital)
data.education = le.fit_transform(data.education)
data.default = le.fit_transform(data.default)
data.housing = le.fit_transform(data.housing)
data.loan = le.fit_transform(data.loan)
data.contact = le.fit_transform(data.contact)
data.month = le.fit_transform(data.month)
data.poutcome = le.fit_transform(data.poutcome)
data.y = le.fit_transform(data.y)

data.isna().any()

age          False
job          False
marital      False
education    False
default      False
balance      False
housing      False
loan         False
contact      False
day          False
month        False
duration     False
campaign     False
pdays        False
previous     False
poutcome     False
y            False
dtype: bool

### **데이터셋 분리**
다음과 같이 feature 변수 x와 label 변수 y를 지정해주고, train_test_split을 이용하여 7:3의 비율로 train data와 test data를 나누었다.

In [4]:
from sklearn.model_selection import train_test_split
y = data['y']
x = data.drop(columns=['y'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)


---



# **3. 실험 설계 및 방법**
## **3-1. DecisionTree**
Decision Tree의 경우, Classification을 하는 데에 있어 가장 대표적이면서도, 직관적인 알고리즘이라고 생각하여 선정하였다. criterion을 gini로 설정하는 것보다 entropy로 설정하였을 때 더 성능이 좋게 나온 것을 확인하여 이후 모델에서도 crition을 entropy로 설정한 후 다른 파라미터들에 변화를 주었다. </br>
max_depth, min_samples_leaf, min_impurity decrease 와 같은 파라미터에 비해 max_leaf_nodes의 파라미터에 변화를 주는 것이 default 모델 대비 성능 향상이 잘 이루어지는 것을 확인할 수 있었다.

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

dt1 = DecisionTreeClassifier(random_state=42)
print('dt1 score: ', cross_val_score(dt1, x_train, y_train, cv=10).mean())

dt2 = DecisionTreeClassifier(criterion="entropy", random_state=42)
print('dt2 score: ', cross_val_score(dt2, x_train, y_train, cv=10).mean())

dt3 = DecisionTreeClassifier(criterion="entropy", min_samples_leaf=4, random_state=42)
print('dt3 score: ', cross_val_score(dt3, x_train, y_train, cv=10).mean())

dt4 = DecisionTreeClassifier(criterion="entropy", max_leaf_nodes=30, random_state=42)
print('dt4 score: ', cross_val_score(dt4, x_train, y_train, cv=10).mean())

dt5 = DecisionTreeClassifier(criterion="entropy", max_leaf_nodes=8, random_state=42)
print('dt5 score: ', cross_val_score(dt5, x_train, y_train, cv=10).mean())

dt6 = DecisionTreeClassifier(criterion="entropy", max_leaf_nodes=8, max_depth = 3, random_state=42)
print('dt6 score: ', cross_val_score(dt6, x_train, y_train, cv=10).mean())

dt7 = DecisionTreeClassifier(criterion="entropy", max_leaf_nodes=30, min_impurity_decrease=0.1, random_state=42)
print('dt7 score: ', cross_val_score(dt7, x_train, y_train, cv=10).mean())


dt1 score:  0.7936033657812243
dt2 score:  0.7965104768190068
dt3 score:  0.8072219105758125
dt4 score:  0.8390202936809107
dt5 score:  0.8435667381620193
dt6 score:  0.8406599571027883
dt7 score:  0.825758125721828


파라미터를 달리한 여러개의 모델 중 가장 성능이 좋게 나온 dt5 모델에 대해서 test-set을 이용한 모델 평가를 하였다.

In [75]:
dt5.fit(x_train, y_train)
print(dt5.score(x_test, y_test))

0.8440677966101695


### **Feature Selection 적용**
feature selection 기법으로 **'반복적 특성 선택**'을 사용하였다. 
특성 선택에 사용한 모델은 Random Forest 이며, n_features_to_select=7로 설정하여 feature이 7개가 남을 때까지 중요도가 낮은 특성을 제거하도록 하였다. 여기서 select 된 feature들은 아래 다른 알고리즘들의 Feature Selection 전, 후 성능을 비교할 때에도 사용 될 예정이다.

In [57]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

select = RFE(RandomForestClassifier(n_estimators=100, random_state=42), n_features_to_select=8)
select.fit(x_train, y_train)

x_new_train = select.transform(x_train)
x_new_test = select.transform(x_test)

feature selection을 거쳐서 새롭게 만들어진 data set에 대해서 모델 평가를 한다.

In [58]:
dt5.fit(x_new_train, y_train)
print(dt5.score(x_new_test, y_test))

0.8440677966101695


## **3-2. RandomForest**
위에서 사용한 Decision Tree에서 더 나아가 여러개의 결정트리를 만들어 그 평균 예측치를 사용하면 어느 정도의 성능 향상이 일어날지 확인하기 위해 앙상블 대표 기법인 Random Forest를 사용해 보았다. 몇 개의 결정 트리를 사용할 것인지를 결정하는 n_estimators 파라미터와 위의 Decision Tree 실험에서 성능에 영향을 가장 많이 준 max_leaf_nodes 파라미터를 위주로 변화를 주면서 실험을 진행하였다.

In [63]:
from sklearn.ensemble import RandomForestClassifier

rf1 = RandomForestClassifier(criterion="entropy", random_state=42)
print('rf1 score: ', cross_val_score(rf1, x_train, y_train, cv=10).mean())

rf2 = RandomForestClassifier(criterion="entropy", max_leaf_nodes=100, random_state=42)
print('rf2 score: ', cross_val_score(rf2, x_train, y_train, cv=10).mean())

rf3 = RandomForestClassifier(criterion="entropy", n_estimators=150, random_state=42)
print('rf3 score: ', cross_val_score(rf3, x_train, y_train, cv=10).mean())

rf4 = RandomForestClassifier(criterion="entropy", n_estimators=150, max_depth = 20, random_state=42)
print('rf4 score: ', cross_val_score(rf4, x_train, y_train, cv=10).mean())

rf1 score:  0.8466569872958256
rf2 score:  0.8455667381620193
rf3 score:  0.8477469064510806
rf4 score:  0.8475647582907111


파라미터를 달리한 여러개의 모델 중 가장 성능이 좋게 나온 rf3 모델에 대해서 test-set을 이용한 모델 평가를 하였다.

In [71]:
rf3.fit(x_train, y_train)
print(rf3.score(x_test, y_test))

0.8567796610169491


### **Feature Selection 적용**

In [65]:
rf3.fit(x_new_train, y_train)
print(rf3.score(x_new_test, y_test))

0.8597457627118644


## **3-3. MLP**
실험 데이터의 경우 feature 수가 16개이기 때문에 복잡도가 높고, linearity로 표현할 수 없다. MLP는 여러 layer를 사용하기 때문에 Decision Tree에 비해 경계선이 유연하여 데이터의 non-linearity를 더 잘 표현하고, 그에 따라 classification 성능도 더 좋을 것이라고 생각하여 선정하게 되었다. MLP에서는 hidden layer의 개수를 결정하는 것이 가장 중요하기 때문에 default 모델에 비해 hidden layer 개수를 더 많이 해서 정확도를 높이고자 하였다. 또한 learning_rate와 max_iter의 파라미터를 변경하면서 최적의 모델을 찾고자 하였다.

In [31]:
from sklearn.neural_network import MLPClassifier

mlp1 = MLPClassifier(random_state=42)
print('mlp1 score: ', cross_val_score(mlp1, x_train, y_train, cv=10).mean())

mlp2 = MLPClassifier(hidden_layer_sizes=200, random_state=42)
print('mlp2 score: ', cross_val_score(mlp2, x_train, y_train, cv=10).mean())

mlp3 = MLPClassifier(hidden_layer_sizes=200, learning_rate_init=0.0005, random_state=42)
print('mlp3 score: ', cross_val_score(mlp3, x_train, y_train, cv=10).mean())

mlp4 = MLPClassifier(hidden_layer_sizes=200, max_iter=300, random_state=42)
print('mlp4 score: ', cross_val_score(mlp4, x_train, y_train, cv=10).mean())

mlp5 = MLPClassifier(hidden_layer_sizes=200, learning_rate_init=0.0005, max_iter=300, random_state=42)
print('mlp5 score: ', cross_val_score(mlp5, x_train, y_train, cv=10).mean())

mlp1 score:  0.7883276687015344
mlp2 score:  0.7714416762910411
mlp3 score:  0.7946906451080681
mlp4 score:  0.7714416762910411
mlp6 score:  0.7946906451080681


파라미터를 달리한 여러개의 모델 중 가장 성능이 좋게 나온 mlp3 모델에 대해서 test-set을 이용한 모델 평가를 하였다.

In [72]:
mlp3.fit(x_train, y_train)
print(mlp3.score(x_test, y_test))

0.7533898305084745


### **Feature Selection 적용**

In [60]:
mlp3.fit(x_new_train, y_train)
print(mlp3.score(x_new_test, y_test))

0.8084745762711865


## **3-4. KNN**
고객 중 성향(feature)이 비슷한 고객들은 정기예금을 들지(label)에 대해 비슷한 결정을 내릴 것이라고 생각하여, 유사한 특징을 가지는 데이터들끼리 모아주는 KNN 알고리즘을 선정하게 되었다. KNN의 경우 몇개의 center를 설정할 것인가가 중요한 요인인데, 몇번의 실험을 통해 default 모델(k가 5)에 비해 k를 10으로 설정한 모델이 더 좋은 성능을 보인다는 것을 확인할 수 있었다. 또한 맨해턴 거리, 유클리디안 거리 등 인스턴스끼리의 거리 측정법도 달리하여 실험하였다.

In [38]:
from sklearn.neighbors import KNeighborsClassifier

knn1 = KNeighborsClassifier()
print('knn1 score: ', cross_val_score(knn1, x_train, y_train, cv=10).mean())

knn2 = KNeighborsClassifier(n_neighbors=10)
print('knn2 score: ', cross_val_score(knn2, x_train, y_train, cv=10).mean())

knn3 = KNeighborsClassifier(p=1)
print('knn3 score: ', cross_val_score(knn3, x_train, y_train, cv=10).mean())

knn4 = KNeighborsClassifier(n_neighbors=10, p=1)
print('knn4 score: ', cross_val_score(knn4, x_train, y_train, cv=10).mean())

knn5 = KNeighborsClassifier(n_neighbors=10, p=1, algorithm='ball_tree')
print('knn5 score: ', cross_val_score(knn5, x_train, y_train, cv=10).mean())

knn1 score:  0.7707101138426002
knn2 score:  0.7781616894901832
knn3 score:  0.7832453390529616
knn4 score:  0.7859722818016829
knn5 score:  0.7861540999835012


파라미터를 달리한 여러개의 모델 중 가장 성능이 좋게 나온 knn5 모델에 대해서 test-set을 이용한 모델 평가를 하였다.

In [73]:
knn5.fit(x_train, y_train)
print(knn5.score(x_test, y_test))

0.7974576271186441


### **Feature Selection 적용**

In [61]:
knn5.fit(x_new_train, y_train)
print(knn5.score(x_new_test, y_test))

0.798728813559322


## **3-5. AdaBoost**
위에서 Decision Tree 알고리즘, Decision Tree에 Bagging 기법을 적용한 Random Forest 알고리즘을 가지고 실험을 진행해보았다. 이번에는 Decision Tree에 Boosting 기법을 적용한 AdaBoost를 선정하였다. Boosting의 경우 올바르게 분류하지 못한 데이터에 더 높은 가중치를 줌으로써 다음 학습에 더 큰 비중을 차지하도록 하므로, 기본 Decision Tree와 Random Forest에 비해 더 좋은 성능을 보일 것으로 예측하였다. Random Forest와 마찬가지로 n_estimators 파라미터와 learning_rate 파라미터를 변경하면서 실험을 진행하였다.

In [44]:
from sklearn.ensemble import AdaBoostClassifier

ada1 = AdaBoostClassifier(n_estimators=10, random_state=42)
print('ada1 score: ', cross_val_score(ada1, x_train, y_train, cv=10).mean())

ada2 = AdaBoostClassifier(n_estimators=50, random_state=42)
print('ada2 score: ', cross_val_score(ada2, x_train, y_train, cv=10).mean())

ada3 = AdaBoostClassifier(n_estimators=50, learning_rate=0.5, random_state=42)
print('ada3 score: ', cross_val_score(ada3, x_train, y_train, cv=10).mean())

ada1 score:  0.8452001319914206
ada2 score:  0.8473826101303414
ada3 score:  0.8510153440026398


파라미터를 달리한 여러개의 모델 중 가장 성능이 좋게 나온 ada3 모델에 대해서 test-set을 이용한 모델 평가를 하였다.

In [74]:
ada3.fit(x_train, y_train)
print(ada3.score(x_test, y_test))

0.8542372881355932


### **Feature Selection 적용**

In [62]:
ada3.fit(x_new_train, y_train)
print(ada3.score(x_new_test, y_test))

0.8563559322033898




---



# **4. 실험 결과 분석**

각 알고리즘 별로, 가장 좋은 성능을 보인 모델에 feature selection을 적용하기 전, 후로 나누어 실험 결과를 분석해보았다.</br>


### **1.   Decision Tree(84.40% -> 84.40%)**
Decision Tree의 경우 알고리즘 자체의 성능은 좋은 편에 속하지만, feature selection을 적용하기 전과 후의 정확도가 둘다 84.40%로 성능 차이가 없었다. 보통 feature selection은 irrelevant하거나 redundant한 feature를 제거해줌으로써 모델의 성능을 높일 수 있도록 해주는데, 이 경우 모든 feature들이 Decision Tree를 구성하는 데에 있어 유의미하게 사용되어 성능 차이가 없는 것으로 생각할 수 있다. </br>
</br>

### **2.   Random Forest(85.67% -> 85.97%)**
Random Forest의 경우 기본 Decision Tree에 비해 약간 더 높은 성능을 보였다. 하나의 결정트리만을 이용하면 data set의 성향에 따라 과적합이나 편중된 예측을 할 가능성이 있는데, Random Forest는 여려개 결정트리의 예측 중 과반수 결과를 사용하기 때문에 위의 위험성을 줄일 수 있다. 그러한 점에서 하나의 Decision Tree에 비해 더 성능이 좋게 나온 것으로 예측한다. 또한 Feature Selection을 적용하였을 때 약간의 성능 향상을 보였다. 여러개의 결정트리의 예측값을 사용하다보면 예측모델 자체가 복잡해질 수 밖에 없는데, 이때 feature space의 복잡도를 줄이는 방법(Feature Selection)이 예측모델의 복잡도를 줄여줌으로써 성능을 높인 것으로 예측할 수 있다. </br>
</br>

### **3.   MLP(75.33% -> 80.84%)**
MLP는 실험에 사용한 모든 모델 중 가장 낮은 정확도를 보였다. 실험에서 cross-validation score을 높이기 위해 hidden layer의 개수를 100개에서 200개로 늘였는데 이로 인해 약간의 과적합이 발생한 것으로 예상한다. 따라서 test set으로 평가했을 때는 75.33%으로 정확도가 가장 낮았지만, feature selection을 적용한 후 다른 모델들에 비해 가장 유의미한 성능 차이를 보였다. 여러 개의 hidden layer을 적용함으로써 생긴 과적합, 복잡성이 feature selection 적용으로 인해 완화된 것으로 예측할 수 있다. </br>
</br>

### **4.   KNN(79.74% -> 79.87%)**
KNN은 79.84% -> 79.87% 의 정확도를 보였다. 다른 모델들에 비해 낮은 성능을 보인 원인은 데이터 셋의 특성에 있다고 생각한다. 실험에 이용 된 데이터 셋의 경우 feature이 16개나 되고, 그 feature들 중에는 데이터 전처리 과정에서 원래 범주형 데이터였던 것을 수치형 데이터로 변환해준 feature들이 다수 존재한다. 이렇게 변환 된 feature들의 값은 범주에 따라 임의로 설정 된 수치값일 뿐이지, 수치값의 차이가 크다(예를 들어, 1과 3의 차이가 보다 1과 7의 차이가 더 크다)고 해서 범주의 차이가 크다는 것을 의미하지는 않는다. 이러한 점에서 범주형 데이터가 많은 데이터에서 인스턴스끼리의 유사도는 측정하기 어려우므로 다른 모델들에 비해 낮은 성능을 보였다고 생각한다. 범주형 데이터를 바이너리 데이터로 변환해주었으면 더 높은 성능을 보였을 것이라고 생각한다. </br>
</br>

### **5.   AdaBoost(85.42% -> 85.63%)**
AdaBoost의 경우 Random Forest와 거의 비슷한 성능을 보였다. AdaBoost는 잘 못 예측한 인스턴스에 더 높은 가중치를 주어 다음 학습 때 이용하므로 성능이 더 좋을 것이라고 예상했는데, 이미 Random Forest가 주어진 데이터 셋에 대하여 높은 성능을 보였기 때문에, 즉 잘 못 예측하는 인스턴스가 적기 때문에 거의 비슷한 성능을 보였다고 생각한다. 오히려 Random Forest에 비해 살짝 낮은 성능을 보이는 것은 n_estimators, learning rate와 같은 파라미터에서 차이가 있었기 때문이라고 예측한다. Feature Selection을 적용하였을 때에는 다른 모델들과 마찬가지로 약간의 성능 향상을 보였고, 모델이 데이터 셋을 학습하는 데 있어 불필요한 feature들을 제거함으로써 성능이 향상된 것으로 보인다.
<br/>
<br/>

---




# **5. 결론**

위 실험에서는 고객의 Bank Marketing에 대한 데이터를 가지고 고객이 정기예금에 가입할 것인지를 예측하는 Classification 모델을 설계하였다. </br>
실험을 진행하기에 앞서 범주형 데이터 -> 수치형 데이터 변환, 무의미한 수치값을 갖는 인스턴스 제거 등을 수행하면서 모델이 데이터를 더 잘 학습할 수 있도록 데이터 전처리 과정을 거쳤다.
Decision Tree, Random Forest, MLP, KNN, AdaBoost 5가지 알고리즘을 사용하여 실험을 진행하였고 각 알고리즘에 대해 파라미터를 변경하면서 높은 성능을 보이는 모델을 선별하였고, 선별 된 모델에 대해서는 feature selection을 적용한 결과값도 도출하였다. 5개의 알고리즘 중에서 Random Forest가 feature selection 적용 전, 후 둘다 가장 높은 성능을 보였다. 범주형 데이터가 많이 포함 된 데이터 셋의 특징을 고려하면 MLP, KNN과 같은 알고리즘보다 Decision Tree가 더 좋은 성능을 낸 결과를 납득할 수 있다. 또한 한가지 Decision Tree보다는 여러개의 Decision Tree의 결과를 종합하는 것이 더 보편적인 예측값을 낼 수 있으므로 위 실험에서 Random Forest가 가장 높은 성능을 보인 결과를 이해할 수 있다.